In [ ]:
# Bloque 01: Mantener la sesión activa
import time
from IPython.display import display, Javascript

# Este código inyecta un script de JavaScript que simula la interacción del usuario
def keep_awake():
    display(Javascript('''
        function ClickConnect() {
            console.log("Manteniendo la conexión activa");
            document.querySelector("colab-toolbar-button#connect").click();
        }
        setInterval(ClickConnect, 60000)
    '''))

keep_awake()

In [ ]:
# Bloque 02: Instalación de paquetes e importación de bibliotecas
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg

import whisper
from google.colab import drive
import os
import shutil
from tqdm.auto import tqdm
import re

In [ ]:
# Bloque 03: Montar Google Drive
drive.mount('/content/drive')

In [ ]:
# Bloque 04: Mover archivos .mp4 de Google Drive a la carpeta /content
ruta_origen = '/content/drive/MyDrive/videos'
ruta_destino = '/content/'

os.makedirs(ruta_destino, exist_ok=True)

for archivo in tqdm(os.listdir(ruta_origen), desc="Copiando archivos MP4"):
    if archivo.endswith('.mp4'):
        origen = os.path.join(ruta_origen, archivo)
        destino = os.path.join(ruta_destino, archivo)
        shutil.copy(origen, destino)

In [ ]:
# Bloque 05: Definir la función de formato de tiempo
def format_timestamp(seconds: float):
    assert seconds >= 0, "Se esperaba una marca de tiempo no negativa"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

# Nueva función para mejorar los subtítulos
def mejorar_subtitulos(subtitulos):
    # Diccionario de términos técnicos y sus equivalentes más formales
    terminos_tecnicos = {
        "red doméstica": "red residencial",
        "configuración de la red": "configuración de red",
        "componentes": "elementos",
        "paso a paso": "de manera sistemática",
        # Agregar más términos según sea necesario
    }

    # Función para reemplazar términos
    def reemplazar_terminos(texto):
        for termino, reemplazo in terminos_tecnicos.items():
            texto = re.sub(r'\b' + re.escape(termino) + r'\b', reemplazo, texto, flags=re.IGNORECASE)
        return texto

    subtitulos_mejorados = []
    for subtitulo in subtitulos.split('\n\n'):
        partes = subtitulo.split('\n', 2)
        if len(partes) == 3:
            numero, tiempo, texto = partes
            # Mejorar el texto
            texto = reemplazar_terminos(texto)
            texto = texto.capitalize()  # Asegurar que cada subtítulo comience con mayúscula
            texto = re.sub(r'\s+', ' ', texto).strip()  # Eliminar espacios extra
            # Reconstruir el subtítulo
            subtitulo_mejorado = f"{numero}\n{tiempo}\n{texto}"
            subtitulos_mejorados.append(subtitulo_mejorado)

    return '\n\n'.join(subtitulos_mejorados)

In [ ]:
# Bloque 06: Carga del modelo de transcripción
model = whisper.load_model("large-v3")

In [ ]:
# Bloque 07: Transcripción de los archivos de video y guardado de los subtítulos en formato SRT

# Crear la carpeta "SubtítulosEN" si no existe
os.makedirs('/content/drive/MyDrive/SubtítulosEN', exist_ok=True)

# Guardar los archivos .srt en la carpeta "SubtítulosEN"
for archivo in tqdm(os.listdir(), desc="Generando subtítulos..."):
    if archivo.endswith(".mp4"):
        result = model.transcribe(archivo)
        subtitulos = ""
        for i, segmento in enumerate(result["segments"], start=1):
            subtitulos += f"{i}\n"
            subtitulos += f"{format_timestamp(segmento['start'])} --> {format_timestamp(segmento['end'])}\n"
            subtitulos += f"{segmento['text'].strip()}\n\n"
        
        # Mejorar los subtítulos
        subtitulos_mejorados = mejorar_subtitulos(subtitulos)
        
        nombre_subtitulos = os.path.splitext(archivo)[0] + ".srt"
        ruta_subtitulos = os.path.join("/content/drive/MyDrive/SubtítulosEN", nombre_subtitulos)
        with open(ruta_subtitulos, "w") as archivo_subtitulos:
            archivo_subtitulos.write(subtitulos_mejorados)

print("Proceso completado. Los archivos .srt mejorados se encuentran en /content/drive/MyDrive/SubtítulosEN")